# 0.0 IMPORTAR BIBLIOTECAS

In [3]:
import datetime
import inflection
import json
import math
import numpy as np
import pandas as pd
import pickle
import random
import requests
import seaborn as sns
import sweetviz as sv
import warnings
import xgboost as xgb

from boruta import BorutaPy
from IPython.core.display import HTML
from IPython.display import Image
from matplotlib import pyplot as plt
from scipy import stats as ss
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from tabulate import tabulate

warnings.filterwarnings( 'ignore' )





In [ ]:
# Ajustar gráficos
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['font.size'] = 10



## 0.1 Funções

In [ ]:
def cramer_v( x, y ):
  cm = pd.crosstab( x, y ).values
  n = cm.sum()
  r, k = cm.shape

  chi2 = ss.chi2_contingency( cm )[0]
  chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
  
  kcorr = k - (k-1)**2/(n-1)
  rcorr = r - (r-1)**2/(n-1)
  return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )

def mean_percentage_error( y, yhat ):
  return np.mean( ( y - yhat ) / y )

def mean_absolute_percentage_error( y, yhat ):
  return np.mean( np.abs( ( y - yhat ) / y ) )

def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    return pd.DataFrame( { 'Model Name': model_name,
    'MAE': mae,
    'MAPE': mape,
    'RMSE': rmse }, index=[0] )

def cross_validation( x_training, kfold,model_name,model):
    mae_list = []
    mape_list = []
    rmse_list = []
    for fold in reversed( range( 1,kfold+1) ):
        print('\nKFold Number: {}'.format( fold ) )
        # start and end date for validation
        validation_start_date = x_training['date'].max() - datetime.timedelta(days=fold*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta(days=(fold-1)*6*7)

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # training and validation dataset

        # training
        xtraining = training.drop( ['date', 'sales'], axis=1 )
        ytraining = training['sales']

        # validation
        xvalidation = validation.drop( ['date', 'sales'], axis=1 )
        yvalidation = validation['sales']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        y_pred = m.predict( xvalidation )

        # performance
        m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1(y_pred ) )

        # store performance of each kfold iteration
        mae_list.append( m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {'Model Name': model_name,
                          'MAE CV': np.round( np.mean( mae_list ), 2 ).astype(str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
                          'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
                          'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str )}, index=[0] )    

## 0.2 Carregar base de dados

In [4]:
df_sales_raw = pd.read_csv( '../datasets/train.csv', low_memory=False )
df_store_raw = pd.read_csv( '../datasets/store.csv', low_memory=False )

# merge entre os dois dataframes
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store' )

# 1.0 DESCRICAO DOS DADOS

In [ ]:
df1 = df_raw.copy()

In [ ]:
# Renomear as colunas para mdoelo snakecase

cols_old = df1.columns
snakecase = lambda x: inflection.underscore( x )
cols_new = list( map( snakecase, cols_old ) )
df1.columns = cols_new

## 1.2. Tamanho dos dados (shape)

In [ ]:
df1.shape

In [ ]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

## 1.3. Tipo dos Dados (Dtypes)

In [ ]:
df1.dtypes

In [ ]:
# Tranaformar a coluna Date para formato de data
df1['date'] = pd.to_datetime( df1['date'] )


## 1.4. Checagem de N/As

In [ ]:
df1.isna().sum()


Como proceder para a retirada dos NAs ?

- Descartar todas as linhas com NA.
- Usando algoritmos de ML para inserir dados.
- Entendendo o negócio para inserir dados.

## 1.5 Preencher os NA

In [ ]:
# competition_distance              
### Para todos os campos NA nesta coluna considerar e inserir o valor de 200.000 (valor máximo escolhido para este df)
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000 if math.isnan(x) else x )

# competition_open_since_month  
### Para todos os campos NA nesta coluna considerar e inserir o valor do mês da venda (Coluna "date").
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'],axis=1)

# competition_open_since_year       
### Para todos os campos NA nesta coluna considerar e inserir o valor do ano da venda (Coluna "date").
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'],axis=1)

# promo2_since_week 
### Para todos os campos NA nesta coluna considerar e inserir o valor da semana da venda (Coluna "date").
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'],axis=1)

# promo2_since_year
### Para todos os campos NA nesta coluna considerar e inserir o valor do ano da venda (Coluna "date").
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'],axis=1)

# promo_interval
month_map = {1: 'Jan',2: 'Fev',3: 'Mar',4: 'Apr',5: 'May',6: 'Jun',7: 'Jul',8: 'Aug', 9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dec'}
df1['promo_interval'].fillna(0, inplace=True) # substituir todos os NA para 0 (zero)
df1['month_map'] = df1['date'].dt.month.map(month_map) # criar uma nova coluna e inserir o mês da coluna "Date" conforme a variavel "month_map"
df1['is_promo'] = df1[['promo_interval','month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)


In [ ]:
df1.sample(5).T

In [ ]:
df1.isna().sum()

## 1.6 Tipos de Mudança

In [ ]:
df1.dtypes

In [ ]:
# transformar valores para inteiros
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype('int64')
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype('int64')
df1['promo2_since_week'] = df1['promo2_since_week'].astype('int64')
df1['promo2_since_year'] = df1['promo2_since_year'].astype('int64')

## 1.7 Descrição Estatística dos dados

Métricas de tendência central : 

- media 
- mediana

Métricas de dispersão : 

- desvio padrão
- valor minimo
- valor máximo
- faixa(range) 
- skew(inclinação)
- curtose 

In [ ]:
numericos = df1.select_dtypes(include=['int64','float64' ])
categoricos = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

### 1.7.1. Atributos Numéricos

In [ ]:
# Métricas de tendência central 
media = pd.DataFrame ( numericos.apply(np.mean)).T
mediana = pd.DataFrame ( numericos.apply(np.median)).T

In [ ]:
# Métricas de dispersão
desvio_padrao = pd.DataFrame ( numericos.apply(np.std)).T
minimo = pd.DataFrame ( numericos.apply(min)).T
maximo = pd.DataFrame ( numericos.apply(max)).T
faixa = pd.DataFrame ( numericos.apply(lambda x: x.max() - x.min())).T
inclinação = pd.DataFrame ( numericos.apply(lambda x: x.skew() )).T
curtose = pd.DataFrame ( numericos.apply(lambda x: x.kurtosis())).T

In [ ]:
#Concatenar
metricas = pd.concat([minimo, maximo, faixa, media, mediana, desvio_padrao, inclinação, curtose ]).T.reset_index()
metricas.columns = ['attributes','min','max','range','mean', 'median', 'std', 'skew','kurtosis' ]
metricas

### 1.7.2. Atributos Categóricos

In [ ]:
# Quantos niveis cada variável categorica possui ?
categoricos.apply( lambda x: x.unique().shape[0] )

In [ ]:
filtro = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.subplot(1, 3, 1)
sns.boxplot(x='state_holiday', y='sales', data=filtro, hue='state_holiday', palette='deep', legend=False)

plt.subplot( 1, 3, 2 )
sns.boxplot( x='store_type', y='sales', data=filtro, hue='store_type', palette='deep', legend=False)

plt.subplot( 1, 3, 3 )
sns.boxplot( x='assortment', y='sales', data=filtro, hue='assortment', palette='deep', legend=False)

# 2.0 FEATURE ENGINEERING

## 2.1 Mapa mental de hipóteses

In [ ]:
Image('../img/mindmap.png')

## 2.2 Criação das Hipoteses

### 2.2.1 Hipoteses da loja

1. Lojas com número maior de funcionários deveriam vender mais.
2. Lojas com maior capacidade de estoque deveriam vender mais.
3. Lojas com maior porte deveriam vender mais.
4. Lojas com maior sortimentos deveriam vender mais.
5. Lojas com competidores mais próximos deveriam vender menos.
6. Lojas com competidores à mais tempo deveriam vendem mais.

### 2.2.2 Hipoteses do produto

1. Lojas que investem mais em Marketing deveriam vender mais.
2. Lojas com maior exposição de produto deveriam vender mais.
3. Lojas com produtos com preço menor deveriam vender mais.
5. Lojas com promoções mais agressivas ( descontos maiores ), deveriam vender mais.
6. Lojas com promoções ativas por mais tempo deveriam vender mais.
7. Lojas com mais dias de promoção deveriam vender mais.
8. Lojas com mais promoções consecutivas deveriam vender mais.



### 2.2.3 Hipoteses do tempo

1. Lojas abertas durante o feriado de Natal deveriam vender mais.
2. Lojas deveriam vender mais ao longo dos anos.
3. Lojas deveriam vender mais no segundo semestre do ano.
4. Lojas deveriam vender mais depois do dia 10 de cada mês.
5. Lojas deveriam vender menos aos finais de semana.
6. Lojas deveriam vender menos durante os feriados escolares.

## 2.3 Lista Final das Hipoteses

1. Lojas com maior sortimentos deveriam vender mais.
2. Lojas com competidores mais próximos deveriam vender menos.
3. Lojas com competidores à mais tempo deveriam vendem mais.
4. Lojas com promoções ativas por mais tempo deveriam vender mais.
5. Lojas com mais dias de promoção deveriam vender mais.
6. Lojas com mais promoções consecutivas deveriam vender mais.
7. Lojas abertas durante o feriado de Natal deveriam vender mais.
7. Lojas deveriam vender mais ao longo dos anos.
9. Lojas deveriam vender mais no segundo semestre do ano.
10. Lojas deveriam vender mais depois do dia 10 de cada mês.
12. Lojas deveriam vender menos aos finais de semana.
12. Lojas deveriam vender menos durante os feriados escolares.

## 2.4 Feature Engineering

In [ ]:
df2 = df1.copy()

In [ ]:
#Ano
df2['year'] = df2['date'].dt.year

#Mês
df2['month'] = df2['date'].dt.month

#Dia
df2['day'] = df2['date'].dt.day

#Dia da Semana
df2['week_of_year'] = df2['date'].dt.isocalendar().week

# Semana no Ano
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

# Data em dias de competição
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'], day= 1), axis=1)
df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

# Data em dias de promoção
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' +df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )
df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply(lambda x: x.days ).astype( int )

# categorias
df2['assortment'] = df2['assortment'].apply( lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )

# feriados
df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day' )


In [ ]:
df2.sample(5).T

# 3.0 Filtragem de Variáveis

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

## 3.1 Filtragem das Linhas

In [ ]:
# Filltro para considerar a coluna Opem diferentes de zero (lojas abertas) e Sales maior que zero (Vendas)
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2 Filtragem das Colunas

In [ ]:
colunas_para_dropar = ['customers', 'open', 'promo_interval','month_map']
df3 = df3.drop(colunas_para_dropar, axis=1)

In [ ]:
df3.columns

# 4.0 EDA - Análise Exploratória de Dados

In [ ]:
df4 = df3.copy()

## 4.1 Análise Univariada

A váriavel resposta para este projeto é a coluna "Sales"

In [ ]:
sns.displot(df4['sales'], kde=False)

### 4.1.2 Variável numérica

In [ ]:
numericos.hist(bins=25);

- Store : Numeração de cada loja, grafico pouco explicativo
- Day of week : Valores de vendas parecidos em todos os dias
- Sales : Valores de venda concentrado em até 20k
- Customers : Valres da quantidade de clientes/dia concentrado até em 2k
- Open : Maioria das lojas abertas, valor =1
- Promo : Maioria das vendas em dias de não promoção, valor=0
- SchoolHoliday : Maioria das vendas em dias de aulas normais(sem férias), valor=0
- CompetitionDistance : Valres da quantidade de distencias concentrado até em 2k
- competition_open_since_month : maiores vendas em Setembro, Abril e Novembro.Piores em Janeiro e Agosto.
- competition_open_since_year : maiores vendas no último ano
- promo2: valores muito parecidos, pouco relevante
- promo2_since_week : comportamento do grafico se parece um pouco com o competition_open_since_month
- promo2_since_year : aqui as maiores vendas sao concentradas em 2013
- is_promo : maioria com valor = 0



In [ ]:
report = sv.analyze(numericos)
report.show_notebook()

### 4.1.2 Variável categórica

In [ ]:
categoricos.head()

In [ ]:
# state_holiday
a = df4[df4['state_holiday'] != 'regular_day']
sns.countplot(x=a['state_holiday'], data=a, hue='state_holiday', palette='deep', legend=False)

In [ ]:
# state_holiday
sns.kdeplot( df4[df4['state_holiday'] == 'public_holiday']['sales'],label='public_holiday', fill=True )
sns.kdeplot( df4[df4['state_holiday'] == 'easter_holiday']['sales'],label='easter_holiday', fill=True )
sns.kdeplot( df4[df4['state_holiday'] == 'christmas']['sales'],label='christmas', fill=True )

In [ ]:
# store_type
sns.countplot(x=df4['store_type'],hue=df4['store_type'], palette='deep')

In [ ]:
# store_type
sns.kdeplot( df4[df4['store_type'] == 'a']['sales'],label='a', fill=True )
sns.kdeplot( df4[df4['store_type'] == 'b']['sales'],label='b', fill=True )
sns.kdeplot( df4[df4['store_type'] == 'c']['sales'],label='c', fill=True )
sns.kdeplot( df4[df4['store_type'] == 'd']['sales'],label='d', fill=True )
plt.legend(title='Store Type', loc='upper right')

In [ ]:
# assortment
sns.countplot(x=df4['assortment'], hue=df4['assortment'], palette='deep', legend=False)

In [ ]:
# assortment
sns.kdeplot( df4[df4['assortment'] == 'basic']['sales'],label='basic', fill=True )
sns.kdeplot( df4[df4['assortment'] == 'extended']['sales'],label='extended', fill=True )
sns.kdeplot( df4[df4['assortment'] == 'extra']['sales'],label='extra', fill=True )
plt.legend(title='Store Type', loc='upper right')

## 4.2 Análise Bivariada

#### 1. Lojas com maior sortimentos deveriam vender mais.

FALSO = O "Extra" (maior sortimento) vendem menos.

- De acordo com o primeiro gráfico, a loja EXTRA é a que menos vende.
- De acordo com o segundo gráfico, as lojas BASIC e EXTENDED possuem variação de mensal mensal muito parecidos de uma com a outra.
- De acordo com o segundo gráfico, a variação de vendas das lojas EXTRA nãoé linear como mostra os segundo gráfico.

In [ ]:
# total de vendas por sortimento
h1 = df4[['assortment','sales']].groupby('assortment').sum().reset_index()
sns.barplot(h1, x='assortment', y='sales',hue='assortment', palette='deep', legend=False)


# grafico em linha de vendas mensal
aux2 = df4[['year_week', 'assortment', 'sales']].groupby(['year_week','assortment'] ).sum().reset_index()
aux2.pivot( index='year_week', columns='assortment', values='sales' ).plot()

# grafico em linha de vendas mensal apenas a linha "extra"
aux3 = aux2[aux2['assortment'] == 'extra']
aux3.pivot( index='year_week', columns='assortment', values='sales' ).plot()

#### 2. Lojas com competidores mais próximos deveriam vender menos.

FALSO = Lojas mais próximas vendem mais

- Maior quantidade de vendas são as lojas com distâncias de até 1 Km

In [ ]:
h2 =  df4[['competition_distance', 'sales']].groupby( 'competition_distance' ).sum().reset_index()
sns.barplot(h2, x='competition_distance', y='sales')

In [ ]:
#criando bins
bins = list( np.arange( 0, 20000, 1000) )
h2['competition_distance_binned'] = pd.cut( h2['competition_distance'],bins=bins )

h21 = h2[['competition_distance_binned', 'sales']].groupby('competition_distance_binned' ).sum().reset_index()
sns.barplot( x='competition_distance_binned', y='sales', data=h21 )
plt.xticks( rotation=90 );

In [ ]:
#grafico de bolinhas
h2 = df4[['competition_distance', 'sales']].groupby( 'competition_distance' ).sum().reset_index()
sns.scatterplot( x ='competition_distance', y='sales', data=h2 );

In [ ]:
#Força da correlação
sns.heatmap( h2.corr( method='pearson'), annot=True )

#### 3. Lojas com competidores à mais tempo deveriam vendem mais.

FALSO = Lojas com competidores à mais tempo vendem menos

Em competition_time_month, o resultado é a Data da venda - Data da abertura do competidor.
- Valores negativos -> Competidor abriu depois da venda
- Valores positivos -> Competidor abriu antes da venda

O Gráfico informa que quanto mais recente é a competição, maior é a venda

In [ ]:
h3 = df4[['competition_time_month', 'sales']].groupby( 'competition_time_month' ).sum().reset_index()
sns.barplot( h3, x='competition_time_month', y='sales');

In [ ]:
h3 = df4[['competition_time_month', 'sales']].groupby( 'competition_time_month' ).sum().reset_index()
h31 = h3[( h3['competition_time_month'] < 120 ) & (h3['competition_time_month'] != 0 )]
sns.barplot( h31, x='competition_time_month', y='sales', );
plt.xticks( rotation=90 );

In [ ]:
#grafico de regplot
sns.regplot( h31, x ='competition_time_month', y='sales');

In [ ]:
#Força da correlação
sns.heatmap( h3.corr( method='pearson' ), annot=True );

4. Lojas com promoções ativas por mais tempo deveriam vender mais.

FALSO = Lojas com promoções ativas por mais tempo vendem menos

- Valores negativo -> Venda realizada dentro do periodo tradicional de promoção
- Valores positivo -> Venda realizada dentro do periodo de promoção extendida

In [ ]:
h4 = df4[['promo_time_week', 'sales']].groupby( 'promo_time_week' ).sum().reset_index()
sns.barplot( h4, x='promo_time_week', y='sales');

In [ ]:
plt.subplot(2,1,1)
extendido = h4[h4['promo_time_week']>0] # promoção extendida
sns.barplot( extendido, x='promo_time_week', y='sales');
# as vendas caem conforme o tempo de promoção

plt.subplot(2,1,2)
regular = h4[h4['promo_time_week']<0] # promoção regular
sns.barplot( regular, x='promo_time_week', y='sales');

In [ ]:
#grafico de bolinhas
plt.subplot(2,1,1)
sns.regplot( extendido, x='promo_time_week', y='sales');
plt.subplot(2,1,2)
sns.regplot( regular, x='promo_time_week', y='sales');

In [ ]:
#Força da correlação
sns.heatmap( h4.corr( method='pearson' ), annot=True );
# para o heatmap precisa ser passado a feature inteira e não quebrada.

<s>5. Lojas com mais dias de promoção deveriam vender mais.</s>

6. Lojas com mais promoções consecutivas deveriam vender mais.

FALSA = Lojas com mais promoções consecutivas vendem menos

- Verificado que quem participou da promoção extendida (promo2) vendeu menos.

In [ ]:
h5 = df4[['promo','promo2', 'sales']].groupby(['promo','promo2']).sum().sort_values('sales', ascending=False).reset_index()
h5


In [ ]:
# Verifiar comportamento durante o tempo

aux1 = df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 1 )][['year_week','sales']].groupby( 'year_week' ).sum().reset_index()
ax = aux1.plot()

aux2 = df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 0 )][['year_week','sales']].groupby( 'year_week' ).sum().reset_index()

aux2.plot( ax=ax )

ax.legend( labels=['Tradicional & Extendida', 'Apenas Extendida']);

7. Lojas abertas durante o feriado de Natal deveriam vender mais.

FALSA = Lojas abertas durante o feriado de Natal não vendem mais do que os outros feriados

- Dentro os feriadados,o feriado de Natal é o que menos vende, em nenhum ano.

In [ ]:
a1 = df4[df4['state_holiday'] != 'regular_day']
h7 = a1[['state_holiday', 'sales']].groupby( 'state_holiday' ).sum().reset_index()
sns.barplot( h7, x='state_holiday', y='sales');

In [ ]:
a2 = a1[['year', 'state_holiday', 'sales']].groupby( ['year','state_holiday']).sum().reset_index()
sns.barplot( a2, x='year', y='sales', hue='state_holiday');

8. Lojas deveriam vender mais ao longo dos anos.

FALSO = Lojas vendem menos ao logo dos anos.

- As vendas vem caindo durante 2013 até 2015

In [ ]:
h8 = df4[['year', 'sales']].groupby( ['year',]).sum().reset_index()
sns.barplot( h8, x='year', y='sales');

In [ ]:
sns.regplot( h8, x='year', y='sales');

In [ ]:
sns.heatmap( h8.corr( method='pearson' ), annot=True );

9. Lojas deveriam vender mais no segundo semestre do ano.

FALSO = As lojas vemde menos no segundo semestre.

- A partir de Agosto as vendam caem drasticamente.

In [ ]:
h9 = df4[['month', 'sales']].groupby( ['month']).sum().reset_index()
sns.barplot( h9, x='month', y='sales');

In [ ]:
sns.regplot( h9, x='month', y='sales');

In [ ]:
sns.heatmap( h9.corr( method='pearson' ), annot=True );

10 Lojas deveriam vender mais depois do dia 10 de cada mês.

*VERDADEIRA* - Bem óbvio porque depois do dia dia temos mais 20 dias para fechar o mês.

In [ ]:
h10 = df4[['day', 'sales']].groupby( ['day']).sum().reset_index()
sns.barplot( h10, x='day', y='sales');

In [ ]:
sns.regplot( h10, x='day', y='sales');

In [ ]:
sns.heatmap( h10.corr( method='pearson'), annot=True );

11. Lojas deveriam vender menos aos finais de semana.

*VERDADEIRA* - Situação parecida como acima, visto que fim de semana são apenas dois dias da semana.

In [ ]:
h11 = df4[['day_of_week', 'sales']].groupby( ['day_of_week']).sum().reset_index()
sns.barplot( h11, x='day_of_week', y='sales');

In [ ]:
sns.regplot( h11, x='day_of_week', y='sales');

In [ ]:
sns.heatmap( h11.corr( method='pearson' ), annot=True );

In [ ]:
h10['antes_depois'] = h10['day'].apply(lambda x: 'antes_dia_10' if x <=10 else 'depois_dia_10')
h111 = h10[['antes_depois','sales']].groupby( 'antes_depois').sum().reset_index()
sns.barplot( h111, x='antes_depois', y='sales');

12. Lojas deveriam vender menos durante os feriados escolares.

*VERDADEIRA* - Pois existem muito mais dias regulares do que dias com feriados escolares.

In [ ]:
h12 = df4[['school_holiday', 'sales']].groupby( ['school_holiday']).sum().reset_index()
sns.barplot( h12, x='school_holiday', y='sales');

In [ ]:
# Vendas por mês
h121 =df4[['school_holiday', 'month', 'sales']].groupby( ['month','school_holiday']).sum().reset_index()
sns.barplot( h121, x='month', y='sales', hue='school_holiday');

In [ ]:
sns.heatmap( h12.corr( method='pearson' ), annot=True );

### 4.2.1 Resumo das hipóteses

In [ ]:
tab =[['Hipoteses', 'Conclusao', 'Relevancia'],
        ['H1', 'Falsa', 'Baixa'],
        ['H2', 'Falsa', 'Media'],
        ['H3', 'Falsa', 'Media'],
        ['H4', 'Falsa', 'Baixa'],
        ['H5', '-', '-'],
        ['H6', 'Falsa', 'Baixa'],
        ['H7', 'Falsa', 'Media'],
        ['H8', 'Falsa', 'Alta'],
        ['H9', 'Falsa', 'Alta'],
        ['H10', 'Verdadeira', 'Alta'],
        ['H11', 'Verdadeira', 'Alta'],
        ['H12', 'Verdadeira', 'Baixa'],
]
print( tabulate( tab, headers='firstrow' ) )

### 4.3.1 Atributos Numéricos

In [ ]:
correlacao = numericos.corr(method='pearson')
sns.heatmap(correlacao, annot=True)

# quanto mais escuro, maior a correlação negativa.
# quanto mais claro, maior a correlação positiva.

### 4.3.2 Atributos Categóricos

In [ ]:
a = df4.select_dtypes(include='object')

In [ ]:
# Calculate cramer V
a1 = cramer_v( a['state_holiday'], a['state_holiday'] )
a2 = cramer_v( a['state_holiday'], a['store_type'] )
a3 = cramer_v( a['state_holiday'], a['assortment'] )
a4 = cramer_v( a['store_type'], a['state_holiday'] )
a5 = cramer_v( a['store_type'], a['store_type'] )
a6 = cramer_v( a['store_type'], a['assortment'] )
a7 = cramer_v( a['assortment'], a['state_holiday'] )
a8 = cramer_v( a['assortment'], a['store_type'] )
a9 = cramer_v( a['assortment'], a['assortment'] )

In [ ]:
# Final dataset
d = pd.DataFrame( {'state_holiday': [a1, a2, a3], 'store_type': [a4, a5, a6], 'assortment': [a7, a8, a9] })
d = d.set_index( d.columns )
sns.heatmap( d, annot=True )

# 5.0 PREPARAÇÃO DOS DADOS

In [ ]:
df5 = df4.copy()

## 5.1 Normalização

Normalização =  (Variavel - Média) / desvio padrão

Como encontrar as melhores variaveis ? 

- Nos resultados da 4.1 Analise univariada e 4.1.2 Variável numérica

## 5.2 Rescaling

Min-Max Scaler = (Variavel - Média) / Valor Máximo - Valor Minimo

Robust Scaler = (Variavel - Média) / Terceiro Quartil - Primeiro Quartil (quando temos outliers)

In [ ]:
a = df5.select_dtypes(include=['int64','float64'])

In [ ]:
a.head()

In [ ]:
# competition_distance (Verificado no boxplot = possui muitos outliers)
sns.boxplot(df5['competition_distance'])

In [ ]:
# Como temos muitos outliers, usaremos o robust scaler
rs = RobustScaler()
df5['competition_distance'] = rs.fit_transform(df5[['competition_distance']].values)

In [ ]:
# competition_time_month (Verificado no boxplot = possui muitos outliers)
sns.boxplot(df5['competition_time_month'])

In [ ]:
# Como temos muitos outliers, usaremos o robust scaler
rs = RobustScaler()
df5['competition_time_month'] = rs.fit_transform(df5[['competition_time_month']].values)

In [ ]:
# promo_time_week (Verificado no boxplot = não possui muitos outliers)
sns.boxplot(df5['promo_time_week'])

In [ ]:
# Como não temos muitos outliers, usaremos o min-max scaler
mms = MinMaxScaler()
df5['promo_time_week'] = mms.fit_transform(df5[['promo_time_week']].values)

In [ ]:
# year 
sns.boxplot(df5['year'])

In [ ]:
mms = MinMaxScaler()
df5['year'] = mms.fit_transform(df5[['year']].values)

In [ ]:
# Criando arquivo PKL
pickle.dump(rs, open("C:/Comunidade_DS/projetos/DS_Producao/parameters/competition_distance_scaler.pkl",'wb'))
pickle.dump(rs, open("C:/Comunidade_DS/projetos/DS_Producao/parameters/competition_time_month_scaler.pkl",'wb'))
pickle.dump(mms, open("C:/Comunidade_DS/projetos/DS_Producao/parameters/promo_time_week_scaler.pkl",'wb'))
pickle.dump(mms, open("C:/Comunidade_DS/projetos/DS_Producao/parameters/year_scaler.pkl",'wb'))

## 5.3 Transformação

### 5.3.1 ENCODIG - Variavel CATEGÓRICA P/ variável NUMÉRICA

One Hot Encoding - Para variáveis de estado

Label Encoding - Para variáveis de nomes

Ordinal Encoding - Segue uma ordenação numérica

Target Encoding  -Para variáveis com muitos niveis

Frequency Encoding

Embedding Encoding

In [ ]:
categoricos.head()

In [ ]:
# state_holiday - Utilizando o one-hot encodingd
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

In [ ]:
# store_type - Utilizando o label encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'].ravel())

In [ ]:
pickle.dump(le, open("C:/Comunidade_DS/projetos/DS_Producao/parameters/store_type_scaler.pkl",'wb'))

In [ ]:
# assortments - Utilizando o ordinal encoding
dicionario = {'basic' : 1, 'extra': 2, 'extended' : 3}
df5['assortment'] = df5['assortment'].map(dicionario)

### 5.3.2 Transformação de Grandeza

Trazer a variavel Resposta "Sales" para o mais proximo possivel de uma distribuição NORMAL

- Transformação Logaritmica
- Box-Cox Transformation
- Cube-Root Transformation
- Square-Root Transformation
- Sine and Cosine Transformation

In [ ]:
sns.histplot( df5['sales'], kde=False )

In [ ]:
# Variavel Resposta é ['sales']
df5['sales']=np.log1p (df5['sales'])
sns.histplot(df5['sales'])

### 5.3.3 Transformação de Natureza

In [ ]:
# day_of_week
df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * ( 2. *np.pi/7 ) ) )
df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos( x * ( 2. *np.pi/7 ) ) )

In [ ]:
# month
df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. *np.pi/12 ) ) )
df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. *np.pi/12 ) ) )

In [ ]:
# day
df5['day_sin'] = df5['day'].apply( lambda x: np.sin( x * ( 2. *np.pi/30 ) ) )
df5['day_cos'] = df5['day'].apply( lambda x: np.cos( x * ( 2. *np.pi/30 ) ) )

In [ ]:
# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin( x * ( 2. *np.pi/52 ) ) )
df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos( x * ( 2. *np.pi/52 ) ) )

# 6.0 SELEÇÃO DE VARIÁVEIS

In [ ]:
df6=df5.copy()

## 6.1 Dataframe de Treino e Teste

In [ ]:
dropadas = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since','competition_since', 'year_week' ]
df6 = df6.drop( dropadas, axis=1 )

In [ ]:
# Dados de treino
x_treino = df6[df6['date'] < '2015-06-19']
y_treino = x_treino['sales']

# Dados de teste
x_teste = df6[df6['date'] >= '2015-06-19']
y_teste = x_teste['sales']

print( 'Data mínima do treino : {}'.format( x_treino['date'].min() ) )
print( 'Data máxima do treino : {}'.format( x_treino['date'].max() ) )

print( '\nData mínima do teste : {}'.format( x_teste['date'].min() ) )
print( 'Data máxima do teste : {}'.format( x_teste['date'].max() ) )

## 6.2 Boruta

In [ ]:
## Dataset de treino e teste do Boruta

#x_treino_n = x_treino.drop( ['date', 'sales'], axis=1 ).values
#y_treino_n = y_treino.values.ravel()

## RandomForestRegressor
#rf = RandomForestRegressor( n_jobs=-1 )

## Boruta
#boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit(x_treino_n, y_treino_n )

### 6.2.1 Colunas Escolhidas pelo Boruta

In [ ]:
#cols_selected = boruta.support_.tolist()

## melhores colunas
#x_treino_fs = x_treino.drop( ['date', 'sales'], axis=1 )
#cols_boruta = x_treino_fs.iloc[:, cols_selected].columns.to_list()

## not selected boruta
#cols_not_selected_boruta = list( np.setdiff1d( x_treino_fs.columns,֒cols_selected_boruta ) )

In [ ]:
# model definition
forest = RandomForestRegressor( n_estimators=100, random_state=0, n_jobs=-1 )

# data preparation
x_treino_n = x_treino.drop( ['date', 'sales'], axis=1 )
y_treino_n = y_treino.values
forest.fit( x_treino_n, y_treino_n )

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
#print the feature ranking
print('Ranking das Variáveis')
df = pd.DataFrame()
for i, j in zip (x_treino, forest.feature_importances_):
    aux = pd.DataFrame({'Variavel:' : i, 'Importancia:': j}, index=[0])
    df = pd.concat([df,aux], axis=0)


print(df.sort_values('Importancia:', ascending=False))

## 6.3 Seleção Manual de Colunas

In [ ]:
cols_boruta = [
'store',
'promo',
'store_type',
'assortment',
'competition_distance',
'competition_open_since_month',
'competition_open_since_year',
'promo2',
'promo2_since_week',
'promo2_since_year',
'competition_time_month',
'promo_time_week',
'day_of_week_sin',
'day_of_week_cos',
'month_sin',
'month_cos',
'day_sin',
'day_cos',
'week_of_year_sin',
'week_of_year_cos']

In [ ]:
# inserir outras colunas
feat_to_add = ['date', 'sales']

# Final Features
cols_boruta_full = cols_boruta.copy()
cols_boruta_full.extend( feat_to_add )

# 7.0 TREINAMENTO DE MACHINE LEARNING

In [ ]:
x_train = x_treino[ cols_boruta ]
x_test = x_teste[cols_boruta ]

# Time Series Data Preparation
x_training = x_treino[ cols_boruta_full ]

## 7.1 Modelo de Média

In [ ]:
aux1 = x_test.copy()
aux1['sales'] = y_teste.copy()

In [ ]:
aux2 = aux1[['store','sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'Predicao'} )
aux1 = pd.merge( aux1, aux2, how='left', on='store' )
y_estimado_baseline = aux1['Predicao']

In [ ]:
#performance
baseline_result = ml_error( 'Modelo de Média', np.expm1( y_teste ), np.expm1(y_estimado_baseline ) )
baseline_result

## 7.2 Modelo de Regressão Linear

In [ ]:
# modelo
lr = LinearRegression().fit(x_train,y_treino)

# prevdição
y_pred_lr = lr.predict(x_test)

# performance
lr_result = ml_error( 'Regressão Linear', np.expm1( y_teste ), np.expm1(y_pred_lr) )
lr_result

### 7.2.1 Modelo de Regressão Linear - Cross Validation

In [ ]:
lr_result_cv = cross_validation( x_training, 5, 'Regressão Linear', lr)
lr_result_cv

## 7.3 LASSO - Modelo de Regressão Linear Regular

In [ ]:
# modelo
lrr = Lasso(alpha=0.01).fit(x_train,y_treino)

# prevdição
y_pred_lrr = lrr.predict(x_test)

# performance
lrr
lrr_result = ml_error( 'Lasso', np.expm1( y_teste ), np.expm1(y_pred_lrr) )
lrr_result

### 7.3.1 LASSO - Modelo de Regressão Linear Regular - Cross Validation

In [ ]:
lrr_result_cv = cross_validation( x_training, 5, 'Lasso', lrr)
lrr_result_cv

## 7.4 Random Forest Regressor

In [ ]:
# model
rf = RandomForestRegressor( n_estimators=100, n_jobs=-1, random_state=42 ).fit(x_train,y_treino)

# prediction
y_pred_rf = rf.predict( x_test )
# performance

rf_result = ml_error( 'Random Forest Regressor', np.expm1( y_teste ), np.expm1(y_pred_rf ) )
rf_result

### 7.4.1 Random Forest Regressor - Cross Validation

In [ ]:
rf_result_cv = cross_validation( x_training, 5, 'Random Forest Regressor', rf)
rf_result_cv

## 7.5 XGBoost Regressor

In [ ]:
# modelo
modelo_xgb = xgb.XGBRegressor( objective='reg:squarederror', 
                                n_estimators=100, 
                                #learning_rate=0.01, 
                                max_depth=10, 
                                subsample=0.7,
                                colsample_bytree=0.9  ).fit(x_train,y_treino)
                                
#modelo_xgb = xgb.XGBRegressor(objective='reg:squarederror').fit(x_train,y_treino)

# prevdição
y_pred_xgb = modelo_xgb.predict(x_test)

# performance
xgb_result=ml_error( 'XGBoost Regressor', np.expm1( y_teste ), np.expm1(y_pred_xgb) )
xgb_result

### 7.5.1 XGBoost Regressor - Cross Validation

In [ ]:
xgb_result_cv = cross_validation( x_training, 5, 'XGBoost Regressor', modelo_xgb)
xgb_result_cv

## 7.6 Comparação do resultado de todos os modelos

In [ ]:
modelling_result = pd.concat( [baseline_result, lr_result, lrr_result,rf_result, xgb_result] )
modelling_result.sort_values( 'RMSE' )

## 7.6.2 Real Performance - Cross Validation

In [ ]:
modelling_result_cv = pd.concat( [lr_result_cv, lrr_result_cv,rf_result_cv, xgb_result_cv] )
modelling_result_cv.sort_values( 'RMSE CV' )

# 8.0 FINE TUNING - Random Search

In [ ]:
parametros = {'n_estimators': [100],
              'max_depth': [3,5,9],
              'subsample': [0.1, 0.5,0.75],
              'colsample_bytree': [0.3, 0.5, 0.9],
              'min_child_weight' : [3,8,15]}

MAX_EVAL = 15

In [ ]:
resultado_final = pd.DataFrame()

for i in range(MAX_EVAL):
    lista = { k: random.sample( v, 1 )[0] for k, v in parametros.items() }
    print( lista )

    modelo_xgb = xgb.XGBRegressor( objective='reg:squarederror', 
                                n_estimators=lista['n_estimators'], 
                                max_depth=lista['max_depth'], 
                                subsample=lista['subsample'],
                                colsample_bytree=lista['colsample_bytree'],
                                min_child_weight=lista['min_child_weight'])


    resultado = cross_validation( x_training, 5, 'XGBoost Regressor', modelo_xgb)
    resultado_final = pd.concat( [resultado_final, resultado] )

resultado_final

## Modelo Final

In [ ]:
parametros_final = {'n_estimators': 100,
              'max_depth': 9,
              'subsample': 0.75,
              'colsample_bytree': 0.5,
              'min_child_weight' : 8}

In [ ]:
# modelo 
modelo_xgb_tunado = xgb.XGBRegressor( objective='reg:squarederror', 
                                n_estimators=parametros_final['n_estimators'], 
                                max_depth=parametros_final['max_depth'], 
                                subsample=parametros_final['subsample'],
                                colsample_bytree=parametros_final['colsample_bytree'],
                                min_child_weight=parametros_final['min_child_weight']).fit(x_train,y_treino)

# predição
y_pred_xgb_tunado = modelo_xgb_tunado.predict(x_test)

# performance
xgb_result_tunado = ml_error( 'XGBoost Regressor', np.expm1( y_teste ), np.expm1(y_pred_xgb_tunado) )
xgb_result_tunado

# 9.0 TRADUÇÃO E INTERPRETAÇÃO DO ERRO

In [ ]:
df9 = x_teste[cols_boruta_full]

# Ajustar para escala original (rescale)
df9['sales'] = np.expm1(df9['sales'])
df9['predictions'] = np.expm1(y_pred_xgb_tunado)

## 9.1 Performance de Negócio

In [ ]:
# Soma das predições
soma = df9[['store','predictions']].groupby('store').sum().reset_index()

# MAE e MAPE
df9_mae = df9[['store','sales','predictions']].groupby('store').apply(lambda x: mean_absolute_error (x['sales'],x['predictions'] )).reset_index().rename(columns={0:'MAE'})
df9_mape = df9[['store','sales','predictions']].groupby('store').apply(lambda x: mean_absolute_percentage_error (x['sales'],x['predictions'] )).reset_index().rename(columns={0:'MAPE'})

# Merge
df9_merge = pd.merge(df9_mae,df9_mape,how='inner', on='store')
df_mergeado = pd.merge(soma,df9_merge,how='inner', on='store')

# Cenários
df_mergeado['pior_cenario'] = df_mergeado['predictions'] - df_mergeado['MAE']
df_mergeado['melhor_cenario'] = df_mergeado['predictions'] + df_mergeado['MAE']

#Ordenação Colunas
df_mergeado = df_mergeado[['store','predictions', 'pior_cenario', 'melhor_cenario', 'MAE', 'MAPE']]

In [ ]:
df_mergeado.sort_values('MAPE', ascending=True).head(10)

In [ ]:
sns.scatterplot( x='store', y='MAPE', data=df_mergeado )

## 9.2 Performance Total

In [ ]:
total = df_mergeado[['predictions', 'pior_cenario', 'melhor_cenario']].apply( lambda x:np.sum( x ), axis=0 ).reset_index().rename( columns={'index': 'Scenario', 0:'Values'} )
total['Values'] = total['Values'].map( 'R${:,.2f}'.format )
total

## 9.3 Performance Machine Learning

In [ ]:
df9['error'] = df9['sales'] - df9['predictions']
df9['error_rate'] = df9['predictions'] / df9['sales']

In [ ]:
plt.subplot( 2, 2, 1 )
sns.lineplot( x='date', y='sales', data=df9, label='SALES' )
sns.lineplot( x='date', y='predictions', data=df9, label='PREDICTIONS' )

plt.subplot( 2, 2, 2 )
sns.lineplot( x='date', y='error_rate', data=df9 )
plt.axhline( 1, linestyle='--')

plt.subplot( 2, 2, 3 )
sns.distplot( df9['error'] )

plt.subplot( 2, 2, 4 )
sns.scatterplot(x=df9['predictions'],y=df9['error'])

# 10.0. DEPLOY PARA PRODUÇÃO

In [ ]:
# modelo treinado
pickle.dump(modelo_xgb_tunado, open("C:/Comunidade_DS/projetos/DS_Producao/model/modelo_rossmann.pkl",'wb'))

## 10.1 Rossmann Class

In [ ]:
import pickle
import pandas as pd
import inflection
import numpy as pd
import math
import datetime

class Rossmann (object):
    def __init__ (self):
        self.home.path = "C:/Comunidade_DS/projetos/DS_Producao/"
        self.competition_distance_scaler = pickle.load(open(self.home_path + 'parameters/competition_distance_scaler.pkl', 'rb'))
        self.competition_time_month_scaler = pickle.load(open(self.home_path + 'parameters/competition_time_month_scaler.pkl', 'rb'))
        self.promo_time_week_scaler = pickle.load(open(self.home_path + 'parameters/promo_time_week_scaler.pkl', 'rb'))
        self.year_scaler = pickle.load(open(self.home_path + 'parameters/year_scaler.pkl', 'rb'))
        self.store_type_scaler = pickle.load(open(self.home_path + 'parameters/store_type_scaler.pkl', 'rb'))

    def limpeza_dados(self,df1):
        colunas_original = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo',
                            'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
                            'CompetitionDistance', 'CompetitionOpenSinceMonth',
                            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
                            'Promo2SinceYear', 'PromoInterval']
                            
        snakecase = lambda x: inflection.underscore(x) 
        colunas_novas = list(map(snakecase, colunas_original))
        df1.columns = colunas_novas

        df1['date'] = pd.to_datetime( df1['date'] )

        df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000 if math.isnan(x) else x )       
        df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'],axis=1)
        df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'],axis=1)
        df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'],axis=1)
        df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'],axis=1)
        month_map = {1: 'Jan',2: 'Fev',3: 'Mar',4: 'Apr',5: 'May',6: 'Jun',7: 'Jul',8: 'Aug', 9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dec'}
        df1['promo_interval'].fillna(0, inplace=True)
        df1['month_map'] = df1['date'].dt.month.map(month_map)
        df1['is_promo'] = df1[['promo_interval','month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

        df1['competition_open_since_month'] = df1['competition_open_since_month'].astype('int64')
        df1['competition_open_since_year'] = df1['competition_open_since_year'].astype('int64')
        df1['promo2_since_week'] = df1['promo2_since_week'].astype('int64')
        df1['promo2_since_year'] = df1['promo2_since_year'].astype('int64')

        return df1


    def feature_engineering(self,df2):

        df2['year'] = df2['date'].dt.year
        df2['month'] = df2['date'].dt.month
        df2['day'] = df2['date'].dt.day
        df2['week_of_year'] = df2['date'].dt.isocalendar().week
        df2['year_week'] = df2['date'].dt.strftime('%Y-%W')
        df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'], day= 1), axis=1)
        df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)
        df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' +df2['promo2_since_week'].astype(str)
        df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ) )
        df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply(lambda x: x.days ).astype( int )
        df2['assortment'] = df2['assortment'].apply( lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )
        df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day' )

        df2 = df2[(df2['open'] != 0)] 
        colunas_para_dropar = ['open', 'promo_interval','month_map']
        df2 = df2.drop(colunas_para_dropar, axis=1)

        return df2
    
    def preparacao (self,df5):
        
        df5['competition_distance'] = self.competition_distance.scaler.fit_transform(df5[['competition_distance']].values)
        df5['competition_time_month']= self.competition_time_month_scaler.fit_transform(df5[['competition_time_month']].values)
        df5['promo_time_week']= self.promo_time_week_scaler.fit_transform(df5[['promo_time_week']].values)
        df5['year']= self.year_scaler.fit_transform(df5[['year']].values)

        df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])
        df5['store_type'] = self.store_type_scaler.fit_transform(df5['store_type'])

        dicionario = {'basic':1, 'extra':2,'extended':3 }
        df5['assortment'] = df5['assortment'].map(dicionario)

        df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * ( 2. *np.pi/7 ) ) )
        df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos( x * ( 2. *np.pi/7 ) ) )

        df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. *np.pi/12 ) ) )
        df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. *np.pi/12 ) ) )

        df5['day_sin'] = df5['day'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
        df5['day_cos'] = df5['day'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

        # week of year
        df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
        df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )

        colunas_escolhidas = ['store',
                            'promo',
                            'store_type',
                            'assortment',
                            'competition_distance',
                            'competition_open_since_month',
                            'competition_open_since_year',
                            'promo2',
                            'promo2_since_week',
                            'promo2_since_year',
                            'competition_time_month',
                            'promo_time_week',
                            'day_of_week_sin',
                            'day_of_week_cos',
                            'month_sin',
                            'month_cos',
                            'day_sin',
                            'day_cos',
                            'week_of_year_sin',
                            'week_of_year_cos']

        return df5[colunas_escolhidas] 
    
    def get_prediction( self, model, original_data, test_data ):
        # prediction
        pred = model.predict( test_data )
        
        # join pred into the original data
        original_data['prediction'] = np.expm1( pred )
        
        return original_data.to_json( orient='records', date_format='iso' ) 

## 10.2 API Handler

In [ ]:
import pickle
import pandas as pd
from flask import Flask, request, Response
from rossmann.rossmann import Rossmann

In [ ]:
# carregar o modelo atual
modelo = pickle.load( open("C:/Comunidade_DS/projetos/DS_Producao/model/modelo_rossmann.pkl", 'rb') )

In [ ]:
# initialize API
app = Flask( __name__ )
@app.route( '/rossmann/predict', methods=['POST'] )

def rossmann_predict():
    test_json = request.get_json()
    
    if test_json: # there is data
        if isinstance( test_json, dict ): # unique example
            test_raw = pd.DataFrame( test_json, index=[0] )

        else: # multiple example
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() )

        # Instantiate Rossmann class
        pipeline = Rossmann()

        # data cleaning
        df1 = pipeline.data_cleaning( test_raw )

        # feature engineering
        df2 = pipeline.feature_engineering( df1 )

        # data preparation
        df3 = pipeline.data_preparation( df2 )

        # prediction
        df_response = pipeline.get_prediction( modelo, test_raw, df3 )

        return df_response
    
    else:
        return Response( '{}', status=200, mimetype='application/json' )

if __name__ == '__main__':
    app.run( '0.0.0.0' , debug=True )

## 10.3 API de Teste

In [5]:
# carregar o dataset de teste
df10 = pd.read_csv( "C:/Comunidade_DS/projetos/DS_Producao/datasets/test.csv" )

In [6]:
# merge test dataset + store
df_test = pd.merge( df10, df_store_raw, how='left', on='Store' )

# choose store for prediction
df_test = df_test[df_test['Store'].isin( [1,2,3] )]

# remove closed days
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()] #lojas diferentes de zero
df_test = df_test.drop( 'Id', axis=1 )

In [7]:
# convert Dataframe to json
data = json.dumps( df_test.to_dict( orient='records' ) )

In [8]:
# API Call

# url = 'http://localhost:5000/rossmann/predict'
# url = 'https://rossmann-api-h4ab.onrender.com/rossmann/predict'
url = 'https://arss-rossmann-api.onrender.com/rossmann/predict'
header = {'Content-type': 'application/json'}
data = data

r = requests.post(url, data = data, headers = header)
print('Status Code {}' .format(r.status_code))

Status Code 200


In [9]:
d1 = pd.DataFrame( r.json(), columns=r.json()[0].keys()  )

In [10]:
d2 = d1[['store', 'prediction']].groupby( 'store' ).sum().reset_index()

for i in range( len( d2 ) ):
    print( 'A loja {} deverá vender R${:,.2f} nas próximas 6 semanas'.format(d2.loc[i, 'store'],d2.loc[i, 'prediction'] ) )

A loja 1 deverá vender R$334,251.89 nas próximas 6 semanas
A loja 3 deverá vender R$249,256.34 nas próximas 6 semanas
